In [10]:
import cobra
import os
from tests import TEST_DIR
import copy

diel_storagepool_model = os.path.join(TEST_DIR, "data", "Diel_AraGEM2010_with_storage_pool.xml")
diel_storagepool_model_2 = cobra.io.read_sbml_model(diel_storagepool_model)
diel_storagepool_model_copy = copy.deepcopy(diel_storagepool_model_2)

In [36]:
for m in diel_storagepool_model_copy.metabolites:
    if m.compartment == "biomass_Night":
        print(m.reactions)

frozenset({<Reaction Bc1_Night at 0x1887da2d0a0>, <Reaction BIO_L_Night at 0x1887da69040>})
frozenset({<Reaction BIO_L_Night at 0x1887da69040>, <Reaction Bc2_Night at 0x1887da2d250>})
frozenset({<Reaction BIO_L_Night at 0x1887da69040>, <Reaction Bc3_Night at 0x1887da2d3d0>})
frozenset({<Reaction Bc4_Night at 0x1887da2d4c0>, <Reaction BIO_L_Night at 0x1887da69040>})
frozenset({<Reaction Bc5_Night at 0x1887da2d5b0>, <Reaction BIO_L_Night at 0x1887da69040>})
frozenset({<Reaction Bc6_Night at 0x1887da2d6a0>, <Reaction BIO_L_Night at 0x1887da69040>})
frozenset({<Reaction Bc7_Night at 0x1887da2d8b0>, <Reaction BIO_L_Night at 0x1887da69040>})
frozenset({<Reaction BIO_L_Night at 0x1887da69040>, <Reaction Bc8_Night at 0x1887da2d9d0>})
frozenset({<Reaction BIO_L_Night at 0x1887da69040>, <Reaction Bc9_Night at 0x1887da2daf0>})
frozenset({<Reaction Bc10_Night at 0x1887da2dc10>, <Reaction BIO_L_Night at 0x1887da69040>})
frozenset({<Reaction Bc11_Night at 0x1887da2dd30>, <Reaction BIO_L_Night at 0x1

In [42]:
reaction = diel_storagepool_model_copy.reactions.get_by_id("BIO_L_Night")

In [52]:
for m in reaction.products:
    print(reaction.get_coefficient(m))

30.0
30.0


In [54]:
print(len(reaction.reactants), len(reaction.metabolites), len(reaction.products))

37 39 2


In [6]:
for m in diel_storagepool_model_copy.metabolites.query("chlorophyl"):
    print(m.reactions)

frozenset({<Reaction R06286_c_Day at 0x1887d819130>})
frozenset({<Reaction R06286_c_Day at 0x1887d819130>, <Reaction R06267_c_Day at 0x1887d8197c0>})
frozenset({<Reaction R03845_c_Day at 0x1887d588fd0>})
frozenset({<Reaction R03845_c_Night at 0x1887dd17400>})
frozenset({<Reaction R06286_c_Night at 0x1887d6fc940>, <Reaction R06267_c_Night at 0x1887d6fc6d0>})
frozenset({<Reaction R06286_c_Night at 0x1887d6fc940>})


In [11]:
for m in diel_storagepool_model_copy.metabolites.query("carotene"):
    print(m.reactions)

frozenset({<Reaction R07856_c_Day at 0x1887d9e9ac0>})
frozenset({<Reaction R06960_c_Day at 0x1887d112b80>, <Reaction R06961_c_Day at 0x1887d08cfa0>})
frozenset({<Reaction R04801_c_Day at 0x1887d5eb610>, <Reaction R07856_c_Day at 0x1887d9e9ac0>, <Reaction R04803_c_Day at 0x1887d5f2f70>})
frozenset({<Reaction R07856_c_Night at 0x1887e12d610>, <Reaction R04801_c_Night at 0x1887d6570a0>, <Reaction R04803_c_Night at 0x1887d657160>})
frozenset({<Reaction R06960_c_Night at 0x1887d737e80>, <Reaction R06961_c_Night at 0x1887d74b040>})
frozenset({<Reaction R07856_c_Night at 0x1887e12d610>})


In [1]:
from cobra import Model, Reaction
from typing import List

class BiomassAdjuster:

    def __init__(self, model: Model, id_biomass_reaction_day: str, id_biomass_reaction_night: str, photosynthesis_reactions_at_night: List[str]) -> None:

        self.model: Model = model
        self.id_biomass_reaction_day: str = id_biomass_reaction_day
        self.id_biomass_reaction_night: str = id_biomass_reaction_night
        self.photosynthesis_reactions_at_night: List[str] = photosynthesis_reactions_at_night

    def reset_boundaries(self) -> None:

        for photosynthesis_reaction in self.photosynthesis_reactions_at_night:
            self.model.reactions.get_by_id(photosynthesis_reaction).lower_bound = 0
            self.model.reactions.get_by_id(photosynthesis_reaction).upper_bound = 0

    def total_biomass_reaction(self) -> None:

        biomass_reaction_day = self.model.reactions.get_by_id(self.id_biomass_reaction_day)
        biomass_reaction_night = self.model.reactions.get_by_id(self.id_biomass_reaction_night)

        biomass_reaction_total = Reaction(id = "Biomass_Total",
                                          name = "Total Biomass Reaction",
                                          subsystem = '',
                                          lower_bound = 0,
                                          upper_bound = 1000)

        for reaction in [biomass_reaction_day, biomass_reaction_night]:
            for metabolite in reaction.metabolites:
                coefficient = reaction.get_coefficient(metabolite.id)
                biomass_reaction_total.add_metabolites({metabolite: coefficient})

        self.model.add_reactions([biomass_reaction_total])
        self.model.objective = biomass_reaction_total

        return biomass_reaction_total

In [3]:
import os
import cobra
from tests import TEST_DIR

diel_model = os.path.join(TEST_DIR, "data", "AraGEM_day_night.xml")
DM = cobra.io.read_sbml_model(diel_model)

test = BiomassAdjuster(DM, "BIO_L_Day", "BIO_L_Night", ["R03845_c_Night", "R07856_c_Night"])

In [4]:
test.reset_boundaries()

In [5]:
test.total_biomass_reaction()

Reaction identifier,Biomass_Total
Name,Total Biomass Reaction
Memory address,0x293439cb7c0
Stoichiometry,0.04 S_4_45_Coumaryl_32_alcohol_biomass_Day + 0.04 S_4_45_Coumaryl_32_alcohol_biomass_Night + 0.0029 S_ATP_biomass_Day + 0.0029 S_ATP_biomass_Night + 30.0 S_ATP_c_Day + 30.0 S_ATP_c_Night + 0.0033... 0.04 4-Coumaryl alcohol_biomass Day + 0.04 4-Coumaryl alcohol_biomass Night + 0.0029 ATP_biomass Day + 0.0029 ATP_biomass Night + 30.0 ATP_c Day + 30.0 ATP_c Night + 0.0033 CTP_biomass Day + 0.0033...
GPR,
Lower bound,0
Upper bound,1000


In [11]:
diel_storagepool_model_2

Name,MODEL1507180028
Memory address,293190a81f0
Number of metabolites,3466
Number of reactions,3202
Number of genes,1404
Number of groups,0
Objective expression,0.01*Ex16_Day - 0.01*Ex16_Day_reverse_efba9
Compartments,"Cytosol Day, Plastid Day, Mitochondrion Day, Peroxisome Day, Vacuole Day, Biomass Day, Accumulation Day, External Day, Cytosol Night, Biomass Night, Plastid Night, Accumulation Night, External Night, Mitochondrion Night, Vacuole Night, Peroxisome Night"


In [91]:
biomass_after = DM.reactions.get_by_id("Biomass_Total")
biomass_before = diel_storagepool_model_copy.reactions.get_by_id("BIO_L_Night")

print(len(biomass_after.reactants), len(biomass_after.products), len(biomass_after.metabolites))
print(len(biomass_before.reactants), len(biomass_before.products), len(biomass_before.metabolites))

74 4 78
37 2 39


In [13]:
str(DM.objective.expression)

'1.0*Biomass_Total - 1.0*Biomass_Total_reverse_a0aa3'